In [1]:
import codecs
import io, json, codecs
import os
import fix

In [2]:
def redif_json(journal_name,redif_file):
    global articles; articles=[]
    global article; article={}
    global creators; creators=[]
    global creator; creator=''
    global e
    for i in content:
        pair=i.split(':')
        j=i.replace(pair[0]+':','').strip()
        key=pair[0].lower()
        if 'template-type' in key:
            if len(article)>0:
                if len(creator)>0:
                    creators.append(creator) 
                    creator=''
                article['ex:creator']=creators
                creators=[]
                if fix.validate(article)==True:
                    articles.append(article)
                else:
                    e=e+1
            article={}
            article['ex:creator']=[]
            article['ex:keyword']=[]
            article['ex:source']=journal_name
            article['ex:year']=''
            article['ex:title']=''
        elif key=='author-name':
            if len(creator)>0:
                creators.append(creator)
            creator=''
            if ',' in j:
                j=' '.join(reversed([part.strip() for part in j.split(',')]))
            creator=j
        elif key=='title':article['ex:title']=j
        elif key=='year':article['ex:date']=j
        elif key=='pages':article['ex:pages']=j
        elif key=='volume':article['ex:volume']=j
        elif key=='issue':article['ex:issue']=j
        elif key=='file-url':article['ex:url']=j
        elif key=='abstract':article['ex:abstract']=j
        elif key=='keywords':
            for keyword in j.replace(';',',').split(','):
                if keyword.strip()!='':
                    article['ex:keyword'].append(keyword)

    if len(article)>0:
        if len(creator)>0:
            creators.append(creator) 
            creator=''
        article['ex:creator']=creators
        creators=[]
        articles.append(article)   
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    if redif_file[-4:]=='.rdf': 
        jsonld_file=jsonld_folder+redif_file.replace('.rdf','.jsonld').replace('/','_')
    if redif_file[-6:]=='.redif': 
        jsonld_file=jsonld_folder+redif_file.replace('.redif','.jsonld').replace('/','_')
    with io.open(jsonld_file, 'w', encoding="utf-8") as f:
        f.write(jd)
    return (jsonld_file)

In [3]:
with open('journals.json') as data_file:    
    journals = json.load(data_file)

redif_folder='redif/'
jsonld_folder='jsonld/'

for journal in journals:
    source=journal['jname']
    journal_name=journal['journal']
    data=journal['data']
    for type_pub in data:   
        file_type=type_pub['type']
        publisher=type_pub['publisher']
        if file_type=='redif':
            print publisher,source,journal_name,
            i=0
            e=0
            for file_name in os.listdir(redif_folder+publisher+'/'+source):
                if file_name[-4:]=='.rdf' or file_name[-6:]=='.redif':
                    redif_file=publisher+'/'+source+'/'+file_name
                    content = []
                    jd=''
                    articles=[]
                    with codecs.open(redif_folder+redif_file,encoding='utf-8',errors='ignore') as file:
                        for line in file:
                            line = line.strip()
                            content.append(line)
                    jsonld_file=redif_json(journal_name,redif_file)
                i=i+1
            print ', NOT VALID=',e,'files=',i

eee aosoci Accounting, Organizations and Society , NOT VALID= 5 files= 1
aea aecrev American Economic Review , NOT VALID= 0 files= 150
ecm emetrp Econometrica , NOT VALID= 0 files= 29
wly emetrp Econometrica , NOT VALID= 0 files= 17
inm orisre Information Systems Research , NOT VALID= 1 files= 1
eee jaecon Journal of Accounting and Economics , NOT VALID= 61 files= 1
bla joares Journal of Accounting Research , NOT VALID= 0 files= 77
kap jbuset Journal of Business Ethics , NOT VALID= 12 files= 394
eee jbvent Journal of Business Venturing , NOT VALID= 0 files= 1
oup jconrs Journal of Consumer Research , NOT VALID= 1082 files= 180
bla jfinan Journal of Finance , NOT VALID= 1692 files= 223
cup jfinqa Journal of Financial and Quantitative Analysis , NOT VALID= 1 files= 52
eee jfinec Journal of Financial Economics , NOT VALID= 72 files= 1
pal jintbs Journal of International Business Studies , NOT VALID= 4 files= 253
bla jomstd Journal of Management Studies , NOT VALID= 1 files= 63
ucp jpolec 